In [ ]:
from typing import Tuple
import numpy as np
import jax.numpy as jnp
from jax import value_and_grad

import utils
import mesher
from material import ThermalMaterial
import bcs
import FE_routines as fe
import mma
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

_Ext = utils.Extent

In [ ]:
bounding_box = mesher.BoundingBox(x=_Ext(0., 40.),
                                  y= _Ext(0., 40.))
nelx, nely = 40, 40 # number of elems along X and Y axis
mesh = mesher.BilinearThermalMesher(nelx = nelx, nely = nely,
                                   bounding_box=bounding_box)

In [ ]:
init_design = np.random.uniform(low=0., high=1., size = (mesh.num_elems, 1))

In [ ]:
material = ThermalMaterial(conductivity=1., heat_capacity=1., mass_density=1.)

In [ ]:
bc = bcs.get_sample_bc(mesh, 
            bcs.SampleThermalBoundaryConditions.LEFT_MID_EDGE_FIXED_ALL_NODE_HEAT)

In [ ]:
fe_solver = fe.FEA(mesh, material, bc)
lossfn = fe_solver.loss_function  # takes in density of shape (num_elems,)

In [ ]:
class Optimizer:
  def __init__(self, lossfn, mesh):
    self.lossfn, self.mesh = lossfn, mesh
  #-----------------------#
  def constraint_fn(self, density: jnp.ndarray,
                        max_vol_frac: float) -> Tuple[jnp.ndarray, jnp.ndarray]:

    def volume_constraint(density:jnp.ndarray)-> float:
      occupied_volume = jnp.einsum('i,i->i',self.mesh.elem_area, density).sum()
      volcons =  occupied_volume/(max_vol_frac*self.mesh.domain_volume) - 1.
      return volcons
    vc, dvc = value_and_grad(volume_constraint)(density.reshape((-1)))

    return jnp.array([vc]).reshape((-1,1)), dvc.reshape((1,-1))
  #-----------------------#
  def objective_fn(self, density: jnp.ndarray)-> Tuple[jnp.ndarray, jnp.ndarray]:
    obj, grad_obj = value_and_grad(self.lossfn)(density.reshape((-1)))
    return jnp.array([obj]), grad_obj.reshape((-1, 1))
  #-----------------------#
  def optimize(self, init_geom, max_vol_frac: float,
     max_iter: int, kkt_tol: float=1e-6, step_tol: float=1e-6,
      move_limit: float=5e-2, plot_interval: int=5):

    self.num_design_var = init_geom.shape[0]
    self.design_var = init_geom
    lower_bound = np.zeros((self.num_design_var, 1))
    upper_bound = np.ones((self.num_design_var, 1))
    mma_params = mma.MMAParams(
        max_iter=max_iter,
        kkt_tol=kkt_tol,
        step_tol=step_tol,
        move_limit=move_limit,
        num_design_var=self.num_design_var,
        num_cons=1,
        lower_bound=lower_bound,
        upper_bound=upper_bound,
    )
    mma_state = mma.init_mma(self.design_var, mma_params)

    def plotfun(x, status = ''):
      plt.figure()
      plt.imshow(x.reshape((self.mesh.nelx, self.mesh.nely)).T, cmap='rainbow')
      plt.title(status)
      plt.show()
    history = {'obj':[], 'vol_cons':[]}
    # MMA Loop

    while not mma_state.is_converged:
      objective, grad_obj = self.objective_fn(mma_state.x)
      constr, grad_cons = self.constraint_fn(mma_state.x, max_vol_frac)
      mma_state = mma.update_mma(
        mma_state, mma_params, objective, grad_obj, constr, grad_cons
      )
      status = (f'epoch {mma_state.epoch:d} obj {objective[0]:.2E} '
                f'vol cons {constr[0,0]:.2E} ')
      history['obj'].append(objective)
      history['vol_cons'].append(constr[0,0])

      
      print(status)
      if mma_state.epoch%plot_interval == 0 or mma_state.epoch==1:
        plotfun(mma_state.x, status)

    return mma_state, history

In [ ]:
optim = Optimizer(lossfn, mesh)

mma_state, history = optim.optimize(init_design, max_vol_frac=0.5,
                                    max_iter=201,
                                    plot_interval=10)

plt.show(block=True)
